## OpenAI Gym Cartpole-v0 ##
https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0  
https://medium.com/@tuzzer/cart-pole-balancing-with-q-learning-b54c6068d947  

In [117]:
import gym
import numpy as np
import random
import math
from time import sleep


## Initialize the "Cart-Pole" environment
env = gym.make('CartPole-v0')

## Defining the environment related constants

# Number of discrete states (bucket) per state dimension
NUM_BUCKETS = (1, 1, 6, 3)  # (x, x', theta, theta')
# Number of discrete actions
NUM_ACTIONS = env.action_space.n # (left, right)
# Bounds for each discrete state
STATE_BOUNDS = list(zip(env.observation_space.low, env.observation_space.high))
STATE_BOUNDS[1] = [-0.5, 0.5]
STATE_BOUNDS[3] = [-math.radians(50), math.radians(50)]

## Creating a Q-Table for each state-action pair
q_table = np.zeros(NUM_BUCKETS + (NUM_ACTIONS,))
print ("Q table shape: ", q_table.shape)

## Learning related constants
MIN_EXPLORE_RATE = 0.01
MIN_LEARNING_RATE = 0.1
MIN_DISCOUNT_FACTOR = 0.96

## Defining the simulation related constants
NUM_EPISODES = 2000
MAX_T = 250
STREAK_TO_END = 500
SOLVED_T = 199
DEBUG_MODE = False


def simulate():

    ## Instantiating the learning related parameters
    learning_rate = get_learning_rate(0)
    explore_rate = get_explore_rate(0)
    discount_factor = get_discount_factor(0)
    #discount_factor = 0.99  # since the world is unchanging

    num_streaks = 0

    for episode in range(NUM_EPISODES):

        # Reset the environment
        obv = env.reset()

        # the initial state
        current_state = state_to_bucket(obv)

        for t in range(MAX_T):
            #env.render()

            # Select an action
            action = select_action(current_state, explore_rate)

            # Execute the action
            obv, reward, done, _ = env.step(action)

            # Observe the result
            next_state = state_to_bucket(obv)

            # Update the Q based on the result
            q_table[current_state + (action,)] = (1 - learning_rate) * q_table[current_state + (action,)] + \
                                                  learning_rate * (reward + discount_factor*np.amax(q_table[next_state]))

            # Setting up for the next iteration
            current_state = next_state

            # Print data
            if (DEBUG_MODE):
                print("\nEpisode = %d" % episode)
                print("t = %d" % t)
                print("Action: %d" % action)
                print("State: %s" % str(state))
                print("Reward: %f" % reward)
                print("Best Q: %f" % best_q)
                print("Explore rate: %f" % explore_rate)
                print("Learning rate: %f" % learning_rate)
                print("Streaks: %d" % num_streaks)

                print("")

            if done:
               print("Episode %d finished after %d time steps" % (episode, t))
               if (t >= SOLVED_T):
                   num_streaks += 1
               else:
                   num_streaks = 0
               break

            #sleep(0.25)

        # It's considered done when it's solved over 120 times consecutively
        if num_streaks > STREAK_TO_END:
            break

        # Update parameters
        explore_rate = get_explore_rate(episode)
        learning_rate = get_learning_rate(episode)
        discount_factor = get_discount_factor(episode)
    
    print ("Finish!!")
    env.render(close=True)


def select_action(state, explore_rate):
    # Select a random action
    if random.random() < explore_rate:
        action = env.action_space.sample()
    # Select the action with the highest q
    else:
        action = np.argmax(q_table[state])
    return action


def get_explore_rate(t):
    return max(MIN_EXPLORE_RATE, min(1, 1.0 - math.log10((t+1)/25)))

def get_learning_rate(t):
    return max(MIN_LEARNING_RATE, min(0.5, 1.0 - math.log10((t+1)/25)))

def get_discount_factor(t):
    return max(MIN_DISCOUNT_FACTOR, min(0.99, 1.0 - math.log10((t+1)/25)))

def state_to_bucket(state):
    bucket_indice = []
    for i in range(len(state)):
        if state[i] <= STATE_BOUNDS[i][0]:
            bucket_index = 0
        elif state[i] >= STATE_BOUNDS[i][1]:
            bucket_index = NUM_BUCKETS[i] - 1
        else:
            # Mapping the state bounds to the bucket array
            bound_width = STATE_BOUNDS[i][1] - STATE_BOUNDS[i][0]
            normalize_state = (state[i]-STATE_BOUNDS[i][0])*(NUM_BUCKETS[i]-1) / bound_width
            bucket_index = int(round(normalize_state))
        bucket_indice.append(bucket_index)
    return tuple(bucket_indice)

if __name__ == "__main__":
    simulate()


[2017-06-12 15:11:50,867] Making new env: CartPole-v0


Q table shape:  (1, 1, 6, 3, 2)
Episode 0 finished after 19 time steps
Episode 1 finished after 14 time steps
Episode 2 finished after 14 time steps
Episode 3 finished after 30 time steps
Episode 4 finished after 10 time steps
Episode 5 finished after 16 time steps
Episode 6 finished after 32 time steps
Episode 7 finished after 25 time steps
Episode 8 finished after 36 time steps
Episode 9 finished after 21 time steps
Episode 10 finished after 16 time steps
Episode 11 finished after 13 time steps
Episode 12 finished after 11 time steps
Episode 13 finished after 13 time steps
Episode 14 finished after 11 time steps
Episode 15 finished after 10 time steps
Episode 16 finished after 33 time steps
Episode 17 finished after 13 time steps
Episode 18 finished after 13 time steps
Episode 19 finished after 23 time steps
Episode 20 finished after 9 time steps
Episode 21 finished after 21 time steps
Episode 22 finished after 32 time steps
Episode 23 finished after 9 time steps
Episode 24 finished 

## A painless Q-learning tutorial ##

A simple implementation of Q-learning,  
refer to this article : http://blog.csdn.net/pi9nc/article/details/27649323  

In [ ]:
import numpy as np

Q = np.zeros((6,6))
R = np.array([[-1,-1,-1,-1,0,-1], 
             [-1,-1,-1,0,-1,100],
             [-1,-1,-1,0,-1,-1],
             [-1,0,-0,-1,0,-1],
             [0,-1,-1,0,-1,100],
             [-1,-0,-1,-1,0,100]])


LEARNING_RATE = 0.8
DEBUG_MODE = False


def get_random_action(state):
    action = np.random.randint(6)
    while R[state][action] == -1:
        action = np.random.randint(6)
    return action


def training():
    for episode in range(100):
        if(DEBUG_MODE):
            print ("EPISODE ", episode)
        
        # get initial state
        state = np.random.randint(5)
        while (state != 5):
            action = get_random_action(state)
            Q[state][action] = R[state][action] + LEARNING_RATE * np.amax(Q[action])
            if(DEBUG_MODE):
                print ("Room %d --> Room %d" % (state, action))
            state = action
            
        if(DEBUG_MODE):
            print ("")

    print ("Q table:")
    print (Q)
    

def verify():
    for episode in range(10):
        print ("EPISODE ", episode)
        state = np.random.randint(5)
        print ("Initial state", state)
        while(state!=5):
            action = np.argmax(Q[state])
            print ("Room %d --> Room %d" % (state, action))
            state = action
        
        print("")

    
if __name__ == "__main__":
    training()
    verify()